Using a already preprocessed IMDB data set, we can use our own preprocessed data set, you can find the csv from the folder if you want

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


dataframe  = pd.read_csv('/content/IMDB_preprocessed_dataset.csv')
df1 = dataframe

In [ ]:
df1.head()

,review,sentiment
0,one reviewers mentioned watching oz episode yo...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically theres family little boy jake thinks...,negative
4,petter matteis love time money visually stunni...,positive


In [ ]:
X = df1['review']
y = df1['sentiment']
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(y.values)
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [ ]:
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

In [ ]:
pip install tabulate

Using Naive bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from tabulate import tabulate

# Define multiple alpha values to test
alpha_values = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

# Create an empty list to store results
results = []

# Iterate over each alpha value
for alpha_value in alpha_values:
    # Create the Multinomial Naive Bayes model with the current alpha value
    clf = MultinomialNB(alpha=alpha_value, fit_prior=False)  # Assuming fit_prior is set to False for consistency

    # Train the model
    clf.fit(X_train_counts, y_train)

    # Make predictions
    y_pred = clf.predict(X_test_counts)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)

    # Append results to the list
    results.append([alpha_value, accuracy])

# Print the results table using tabulate
headers = ["Alpha", "Accuracy"]
print(tabulate(results, headers=headers, tablefmt="grid"))

+----------+------------+
|    Alpha |   Accuracy |
+==========+============+
|    0.001 |     0.8372 |
+----------+------------+
|    0.01  |     0.8455 |
+----------+------------+
|    0.1   |     0.8555 |
+----------+------------+
|    1     |     0.8593 |
+----------+------------+
|   10     |     0.8572 |
+----------+------------+
|  100     |     0.8481 |
+----------+------------+
| 1000     |     0.8088 |
+----------+------------+


Using Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

# Create a Logistic Regression model
logreg_classifier = LogisticRegression()

# Use GridSearchCV to search for the best hyperparameters
grid_search = GridSearchCV(logreg_classifier, param_grid, cv=None, n_jobs=-1)
grid_search.fit(X_train_counts, y_train)

# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Make predictions on the test set using the best model
best_logreg_model = grid_search.best_estimator_
y_pred = best_logreg_model.predict(X_test_counts)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solver 

Best hyperparameters: {'C': 0.1, 'penalty': 'l2'}
Accuracy: 0.89


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Using SVM

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X_train_SVM, X_test_SVM, y_train_SVM, y_test_SVM = train_test_split(X, labels, test_size=0.7, random_state=42)
X_train_counts_SVM = vectorizer.fit_transform(X_train_SVM)
X_test_counts_SVM = vectorizer.transform(X_test_SVM)

# Train the SVM classifier
svm_classifier = SVC(kernel='linear', C=1)
svm_classifier.fit(X_train_counts_SVM, y_train_SVM)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test_counts_SVM)


# Evaluate the model
accuracy = accuracy_score(y_test_SVM, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.85
